<a href="https://colab.research.google.com/github/11kartheek/Qlora_peft/blob/main/merge_and_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q -U git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/Phi-3.5-mini-instruct"


model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [8]:
# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=20)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] What is a large language model? [/INST] A large language model (


In [11]:
from peft import LoraConfig, prepare_model_for_kbit_training, TaskType, PeftModel

In [12]:
peft_model_id = "Kartheekb7/results"
tr_model_id = model_name

model_tr = AutoModelForCausalLM.from_pretrained(tr_model_id,
                                                torch_dtype="auto",
                                                trust_remote_code=True)
model_tr = PeftModel.from_pretrained(model_tr, peft_model_id)
model_tr = model_tr.merge_and_unload()

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/37.8M [00:00<?, ?B/s]

In [13]:
# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model_tr, tokenizer=tokenizer, max_length=20)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


<s>[INST] What is a large language model? [/INST] A large language model is a type of artificial intelligence that is trained on a large amount of text data. It is able to generate human-like text based on the input it receives. Large language models are used in a variety of applications, such as chatbots, virtual assistants, and content generation. They are trained using a technique called deep learning, which involves using multiple layers of neural networks to process and analyze the text data. The training process involves feeding the model large amounts of text data and adjusting the weights of the neural networks to minimize the difference between the model's output and the expected output. Once trained, the model can be used to generate new text based on the input it receives. The output generated by the model is based on the patterns it has learned from the training data, and can be used to create a wide range of text-based applications.

[/INST


In [ ]:
merged_model_id = "Kartheekb7/peft_phi3.5_itr2"
model_tr.push_to_hub(merged_model_id)
tokenizer.push_to_hub(merged_model_id)